In [57]:
import json
import time

from src.experiments.aux_code import *
from datasets.get_datasets import *
import time
from src.brkga.genetic import genetic as brkga
from src.brkga_variation.genetic import genetic as brkga_var
from src.s_genetic.genetic import *
from src.predicate import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
source='worksat'
target='companyeconomicsector'

In [3]:
kb = json.loads(open('src/experiments/kb.txt').readline())

In [4]:
kb_source = kb['yago2s']
kb_target = kb['nell_finances']

In [5]:
yago_str = [['worksat(A, B)',
  {'': 'graduatedfrom(C, B), influences(C, D)',
   'true': 'hasacademicadvisor(E, C), hasacademicadvisor(C, F)',
   'true,false': 'iscitizenof(C, G), iscitizenof(A, G)',
   'true,false,false': 'haswebsite(D, H), diedin(C, I)',
   'false': 'hasacademicadvisor(A, J), hasgender(J, K)',
   'false,false': 'graduatedfrom(L, B), livesin(L, M)',
   'false,false,true': 'iscitizenof(A, M), iscitizenof(L, M)',
   'false,false,false': 'hasacademicadvisor(N, A), graduatedfrom(N, P)'},
  {'true,true': [0.388, 273, 1209],
   'true,false,true': [0.37, 69, 352],
   'true,false,false,true': [7.829, 88, 202],
   'true,false,false,false': [18.136, 687, 631],
   'false,true': [0.451, 865, 344],
   'false,false,true,true': [4.969, 33, 98],
   'false,false,true,false': [17.103, 703, 501],
   'false,false,false,true': [3.67, 85, 16],
   'false,false,false,false': [24.03, 1492, 942]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), influences(D, C)',
   'true': 'hasacademicadvisor(C, E), influences(F, D)',
   'true,false': 'iscitizenof(C, G), iscitizenof(A, G)',
   'true,false,false': 'iscitizenof(A, H), exports(H, I)',
   'false': 'hasacademicadvisor(J, A), hasgender(A, K)',
   'false,true': 'graduatedfrom(L, B), hasacademicadvisor(M, L)',
   'false,false': 'graduatedfrom(A, B)',
   'false,false,false': 'hasacademicadvisor(A, N), hasacademicadvisor(N, P)'},
  {'true,true': [0.378, 283, 1305],
   'true,false,true': [0.385, 79, 353],
   'true,false,false,true': [11.994, 386, 242],
   'true,false,false,false': [17.15, 545, 655],
   'false,true,true': [0.493, 44, 49],
   'false,true,false': [0.36, 417, 76],
   'false,false,true': [0.272, 0, 95],
   'false,false,false,true': [8.587, 290, 100],
   'false,false,false,false': [29.462, 2251, 1420]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), actedin(C, D)',
   'true': 'graduatedfrom(A, B)',
   'true,false': 'wrotemusicfor(E, D), directed(F, D)',
   'false': 'graduatedfrom(G, B), hasacademicadvisor(G, H)',
   'false,true': 'hasacademicadvisor(I, A), graduatedfrom(I, B)',
   'false,true,false': 'livesin(G, J), iscitizenof(A, J)',
   'false,false': 'hasacademicadvisor(K, A), hasgender(A, L)',
   'false,false,false': 'hasacademicadvisor(A, M), hasgender(M, N)'},
  {'true,true': [0.153, 4, 184],
   'true,false,true': [0.414, 433, 1308],
   'true,false,false': [0.49, 77, 67],
   'false,true,true': [0.064, 0, 141],
   'false,true,false,true': [6.93, 67, 220],
   'false,true,false,false': [20.31, 918, 826],
   'false,false,true': [0.362, 427, 79],
   'false,false,false,true': [11.452, 486, 182],
   'false,false,false,false': [27.442, 1883, 1288]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), hasacademicadvisor(D, C)',
   'true': 'graduatedfrom(A, B)',
   'true,false': 'owns(B, E), wasbornin(D, F)',
   'true,false,false': 'hasacademicadvisor(G, A), graduatedfrom(G, B)',
   'false': 'graduatedfrom(A, B)',
   'false,false': 'hasacademicadvisor(A, H), hasgender(H, I)',
   'false,false,true': 'hasacademicadvisor(J, A), graduatedfrom(A, K)',
   'false,false,false': 'graduatedfrom(L, B), haschild(M, L)'},
  {'true,true': [0.146, 5, 384],
   'true,false,true': [0.4, 322, 1131],
   'true,false,false,true': [0.635, 0, 98],
   'true,false,false,false': [18.546, 781, 735],
   'false,true': [0.129, 1, 106],
   'false,false,true,true': [5.613, 243, 38],
   'false,false,true,false': [13.303, 627, 258],
   'false,false,false,true': [10.404, 225, 233],
   'false,false,false,false': [28.122, 2091, 1312]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), actedin(C, D)',
   'true': 'edited(E, D), directed(E, F)',
   'true,false': 'owns(B, G), wasbornin(C, H)',
   'true,false,true': 'iscitizenof(A, I), islocatedin(D, I)',
   'false': 'graduatedfrom(A, B)',
   'false,false': 'graduatedfrom(A, J)',
   'false,false,true': 'owns(B, K), owns(J, L)',
   'false,false,false': 'graduatedfrom(M, B), isknownfor(M, N)'},
  {'true,true': [0.361, 154, 706],
   'true,false,true,true': [3.41, 16, 66],
   'true,false,true,false': [8.413, 145, 170],
   'true,false,false': [0.397, 199, 617],
   'false,true': [0.118, 2, 306],
   'false,false,true,true': [11.773, 283, 306],
   'false,false,true,false': [26.388, 2214, 1128],
   'false,false,false,true': [3.653, 19, 72],
   'false,false,false,false': [22.516, 1263, 924]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(A, B)',
   'true': 'hasgender(A, C), owns(B, D)',
   'false': 'graduatedfrom(E, B), actedin(E, F)',
   'false,true': 'haschild(G, E), wasbornin(E, H)',
   'false,true,true': 'livesin(I, H), hasgender(A, J)',
   'false,true,false': 'wrotemusicfor(K, F), ismarriedto(K, L)',
   'false,false': 'hasacademicadvisor(M, A), graduatedfrom(M, B)',
   'false,false,false': 'hasacademicadvisor(N, A), graduatedfrom(N, P)'},
  {'true,true': [0.163, 4, 183],
   'true,false': [0.111, 2, 307],
   'false,true,true,true': [5.377, 36, 271],
   'false,true,true,false': [1.339, 5, 4],
   'false,true,false,true': [12.385, 220, 730],
   'false,true,false,false': [11.418, 249, 370],
   'false,false,true': [0.096, 0, 95],
   'false,false,false,true': [8.864, 426, 113],
   'false,false,false,false': [35.62, 3353, 2222]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), influences(D, C)',
   'true': 'livesin(D, E), hasacademicadvisor(D, F)',
   'true,false': 'iscitizenof(C, G), iscitizenof(A, G)',
   'true,false,false': 'iscitizenof(A, H), exports(H, I)',
   'false': 'hasacademicadvisor(A, J), hasacademicadvisor(J, K)',
   'false,false': 'graduatedfrom(A, L), owns(L, M)',
   'false,false,true': 'hasacademicadvisor(N, A), iscitizenof(A, P)',
   'false,false,false': 'influences(A, Q), graduatedfrom(R, B)'},
  {'true,true': [0.345, 223, 1101],
   'true,false,true': [0.367, 90, 443],
   'true,false,false,true': [11.432, 399, 242],
   'true,false,false,false': [17.013, 581, 769],
   'false,true': [0.4, 500, 155],
   'false,false,true,true': [2.32, 41, 6],
   'false,false,true,false': [16.611, 802, 459],
   'false,false,false,true': [2.891, 47, 13],
   'false,false,false,false': [24.898, 1612, 1107]}],
 ['worksat(A, B)',
  {'': 'hasacademicadvisor(C, A), graduatedfrom(C, B)',
   'false': 'graduatedfrom(D, B), wrotemusicfor(D, E)',
   'false,true': 'directed(F, E), islocatedin(E, G)',
   'false,true,true': 'diedin(A, H), hascurrency(G, I)',
   'false,true,false': 'hasacademicadvisor(A, J), haswebsite(D, K)',
   'false,false': 'graduatedfrom(L, B), haschild(L, M)',
   'false,false,true': 'iscitizenof(L, N), iscitizenof(A, N)',
   'false,false,false': 'graduatedfrom(A, B)'},
  {'true': [0.085, 0, 241],
   'false,true,true,true': [2.65, 13, 35],
   'false,true,true,false': [6.432, 51, 415],
   'false,true,false,true': [2.016, 5, 50],
   'false,true,false,false': [6.758, 75, 167],
   'false,false,true,true': [4.937, 28, 239],
   'false,false,true,false': [19.173, 777, 1057],
   'false,false,false,true': [1.365, 1, 161],
   'false,false,false,false': [34.069, 3345, 1930]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(A, B)',
   'false': 'graduatedfrom(C, B), haschild(D, C)',
   'false,true': 'iscitizenof(C, E), iscitizenof(A, E)',
   'false,true,true': 'hascurrency(E, F), ismarriedto(G, D)',
   'false,true,false': 'iscitizenof(D, H), hasgender(A, I)',
   'false,false': 'graduatedfrom(A, J), owns(J, K)',
   'false,false,true': 'owns(B, L)',
   'false,false,false': 'graduatedfrom(M, B), owns(B, N)'},
  {'true': [0.13, 6, 490],
   'false,true,true,true': [3.882, 19, 112],
   'false,true,true,false': [5.293, 35, 297],
   'false,true,false,true': [9.176, 133, 478],
   'false,true,false,false': [17.117, 624, 748],
   'false,false,true,true': [10.951, 245, 269],
   'false,false,true,false': [15.82, 927, 360],
   'false,false,false,true': [11.022, 334, 241],
   'false,false,false,false': [26.87, 1972, 1300]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(A, B)',
   'false': 'hasacademicadvisor(A, C), graduatedfrom(D, B)',
   'false,true': 'iscitizenof(D, E), iscitizenof(A, E)',
   'false,true,true': 'dealswith(E, F)',
   'false,true,false': 'hasacademicadvisor(D, G), hasacademicadvisor(G, H)',
   'false,false': 'graduatedfrom(I, B), isknownfor(I, J)',
   'false,false,true': 'influences(K, I), influences(L, A)',
   'false,false,false': 'graduatedfrom(A, M)'},
  {'true': [0.126, 6, 490],
   'false,true,true,true': [3.795, 10, 149],
   'false,true,true,false': [6.43, 75, 199],
   'false,true,false,true': [10.456, 216, 395],
   'false,true,false,false': [8.926, 398, 107],
   'false,false,true,true': [1.073, 1, 39],
   'false,false,true,false': [7.528, 83, 313],
   'false,false,false,true': [27.948, 2231, 1502],
   'false,false,false,false': [23.126, 1275, 1101]}]]

In [6]:
pred_target = create_pred_target(kb_target)

In [22]:
import copy
src_struct = copy.deepcopy(yago_str)
new_src_struct = []
for i in range(0, len(src_struct)):
    new_src_struct.append(define_individual(src_struct[i], i))  
structured_src = src_struct

# target = f'recursion_{target}'

In [31]:
finances_dataset = datasets.load('nell_finances', kb_target, target=target, seed=441773)
train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(finances_dataset, 0)

0


In [33]:
facts = []
facts.extend(train_facts[0])
facts.extend(train_facts[1])
facts.extend(test_facts)
train_facts = [copy.deepcopy(facts), copy.deepcopy(facts)]
test_facts = [copy.deepcopy(facts)]

## Aleatório

In [35]:
start = time.time()
res = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
MELHOR RESULTADO:  -0.4097515
GENERATION:  1
MELHOR RESULTADO:  -0.569688
GENERATION:  2
MELHOR RESULTADO:  -0.590896
GENERATION:  3
MELHOR RESULTADO:  -0.5981645
GENERATION:  4
MELHOR RESULTADO:  -0.598843
GENERATION:  5
MELHOR RESULTADO:  -0.601143
GENERATION:  6
MELHOR RESULTADO:  -0.6022525
GENERATION:  7
MELHOR RESULTADO:  -0.6022525
GENERATION:  8
MELHOR RESULTADO:  -0.6022719999999999
GENERATION:  9
MELHOR RESULTADO:  -0.6022719999999999
GENERATION:  10
MELHOR RES

In [36]:
res

(<src.s_genetic.population.Population at 0x138898c70>,
 [-0.4097515,
  -0.569688,
  -0.590896,
  -0.5981645,
  -0.598843,
  -0.601143,
  -0.6022525,
  -0.6022525,
  -0.6022719999999999,
  -0.6022719999999999,
  -0.6022719999999999,
  -0.6022719999999999,
  -0.6022719999999999,
  -0.6022719999999999,
  -0.6025725],
 [{'m_auc_pr': 0.9861325000000001,
   'm_auc_roc': 0.9873555,
   'm_cll': -0.4097515,
   'm_rec': 0.8639705,
   'm_pred': 0.7200155,
   'm_f1': 0.9003935000000001,
   's_auc_pr': 0.006734500000000032,
   's_auc_roc': 0.005575500000000012,
   's_cll': 0.0009144999999999848,
   's_rec': 0.0036764999999999715,
   's_prec': 0.22008391363352753,
   's_f1': 0.00555650000000002},
  {'m_auc_pr': 0.812988,
   'm_auc_roc': 0.8137945,
   'm_cll': -0.569688,
   'm_rec': 0.6286765000000001,
   'm_pred': 0.6699335,
   'm_f1': 0.719034,
   's_auc_pr': 0.007454999999999989,
   's_auc_roc': 0.006991499999999984,
   's_cll': 0.0010850000000000026,
   's_rec': 0.02941149999999998,
   's_prec': 

In [37]:
def get_k_best_individuals(population, k):
    best_ind = []
    all_fitness = []
    for i in range(0, len(population)):
        all_fitness.append((population[i].fitness.values[0], i))
    
    sorted_ind = sorted(all_fitness, reverse=True, key=lambda tup: tup[0])
    for i in range(0, k):
        best_ind.append(population[sorted_ind[i][1]])
    return best_ind

In [38]:
final_results = {}
final_results[f'{source}->{target}'] = []
final_results[f'{source}->{target}'].append(res)
best = final_results[f'{source}->{target}'][0]
for i in final_results[f'{source}->{target}']:
    if i[1][-1] > best[1][-1]:
        best = i

In [43]:

final_results[f'groot_test:{source}->{target}'] = []
final_results[f'groot_test_rev:{source}->{target}'] = []
final_results[f'inf_res:{source}->{target}'] = []
final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
tr_train_pos = []
tr_train_neg = []
tr_train_facts = []

for i in range(0, len(train_pos)):
    tr_train_pos.extend(train_pos[i])
    tr_train_neg.extend(train_neg[i])
    tr_train_facts.extend(train_facts[i])

train = [tr_train_pos, tr_train_neg, copy.deepcopy(facts)]
test = [test_pos, test_neg, copy.deepcopy(facts)]

individuals = get_k_best_individuals(best[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'yago2s', 'nell_finances')
    res = test_refine_transfer(kb_target, target, refine, transfer, train, test)
    print(res)
    final_results[f'groot_test:{source}->{target}'].append(res)
    import os
#     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
#     thisFile = f'boostsrl/best/test/results_{target}.db'
#     base = os.path.splitext(thisFile)[0]
#     os.rename(thisFile, base + ".txt")
#     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
#     final = []
#     for i in tt:
#         final.append(i.replace('\n', ''))
#     final_results[f'inf_res:{source}->{target}'].append(final)
    print("===========================================================================")
    
    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.719675, 'AUC PR': 0.725288, 'CLL': -0.624744, 'Precision': [0.810811, 0.5], 'Recall': 0.384615, 'F1': 0.521739}, <boostsrl.boostsrl.test object at 0x13832f610>)
INDIVIDUO  2
({'AUC ROC': 0.720533, 'AUC PR': 0.723649, 'CLL': -0.624744, 'Precision': [0.810811, 0.5], 'Recall': 0.384615, 'F1': 0.521739}, <boostsrl.boostsrl.test object at 0x1380da790>)
INDIVIDUO  3
({'AUC ROC': 0.720533, 'AUC PR': 0.723649, 'CLL': -0.624744, 'Precision': [0.810811, 0.5], 'Recall': 0.384615, 'F1': 0.521739}, <boostsrl.boostsrl.test object at 0x138beef40>)


In [44]:
final_results[f'brkga_{source}->{target}'] = []
final_results[f'inf_res_brkga:{source}->{target}'] = []
final_results[f'brkga_var_{source}->{target}'] = []
final_results[f'inf_res_brkga_var:{source}->{target}'] = []
save_results(final_results, source, target, 'yago2s', 'nell_finances')

In [45]:
final_results[f'groot_test:{source}->{target}']

[({'AUC ROC': 0.719675,
   'AUC PR': 0.725288,
   'CLL': -0.624744,
   'Precision': [0.810811, 0.5],
   'Recall': 0.384615,
   'F1': 0.521739},
  <boostsrl.boostsrl.test at 0x13832f610>),
 ({'AUC ROC': 0.720533,
   'AUC PR': 0.723649,
   'CLL': -0.624744,
   'Precision': [0.810811, 0.5],
   'Recall': 0.384615,
   'F1': 0.521739},
  <boostsrl.boostsrl.test at 0x1380da790>),
 ({'AUC ROC': 0.720533,
   'AUC PR': 0.723649,
   'CLL': -0.624744,
   'Precision': [0.810811, 0.5],
   'Recall': 0.384615,
   'F1': 0.521739},
  <boostsrl.boostsrl.test at 0x138beef40>)]

## Guiado

In [47]:
start = time.time()
res = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
MELHOR RESULTADO:  -0.3859575
GENERATION:  1
MELHOR RESULTADO:  -0.387672
GENERATION:  2
MELHOR RESULTADO:  -0.40698449999999997
GENERATION:  3
MELHOR RESULTADO:  -0.4085245
GENERATION:  4
MELHOR RESULTADO:  -0.414719
GENERATION:  5
MELHOR RESULTADO:  -0.41999600000000004
GENERATION:  6
MELHOR RESULTADO:  -0.41999600000000004
GENERATION:  7
MELHOR RESULTADO:  -0.422424
GENERATION:  8
MELHOR RESULTADO:  -0.42307249999999996
GENERATION:  9
MELHOR RESULTADO:  -0.43414949999

In [48]:
res

(<src.s_genetic.population.Population at 0x138df56d0>,
 [-0.3859575,
  -0.387672,
  -0.40698449999999997,
  -0.4085245,
  -0.414719,
  -0.41999600000000004,
  -0.41999600000000004,
  -0.422424,
  -0.42307249999999996,
  -0.43414949999999997,
  -0.43414949999999997,
  -0.43423500000000004,
  -0.4343855,
  -0.437423,
  -0.4395635],
 [{'m_auc_pr': 0.9912615,
   'm_auc_roc': 0.989633,
   'm_cll': -0.3859575,
   'm_rec': 0.935662,
   'm_pred': 0.71078625,
   'm_f1': 0.9278824999999999,
   's_auc_pr': 0.0032885000000000275,
   's_auc_roc': 0.003905999999999965,
   's_cll': 0.005968499999999988,
   's_rec': 0.0018380000000000063,
   's_prec': 0.21393405382076858,
   's_f1': 0.027173500000000017},
  {'m_auc_pr': 0.9886835,
   'm_auc_roc': 0.9868755,
   'm_cll': -0.387672,
   'm_rec': 0.935662,
   'm_pred': 0.7121255,
   'm_f1': 0.9289894999999999,
   's_auc_pr': 0.0025065000000000226,
   's_auc_roc': 0.0029335000000000333,
   's_cll': 0.005860000000000004,
   's_rec': 0.0018380000000000063,
  

In [49]:
final_results = {}
final_results[f'{source}->{target}'] = []
final_results[f'{source}->{target}'].append(res)
best = final_results[f'{source}->{target}'][0]
for i in final_results[f'{source}->{target}']:
    if i[1][-1] > best[1][-1]:
        best = i

In [50]:

final_results[f'groot_test:{source}->{target}'] = []
final_results[f'groot_test_rev:{source}->{target}'] = []
final_results[f'inf_res:{source}->{target}'] = []
final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
tr_train_pos = []
tr_train_neg = []
tr_train_facts = []

for i in range(0, len(train_pos)):
    tr_train_pos.extend(train_pos[i])
    tr_train_neg.extend(train_neg[i])
    tr_train_facts.extend(train_facts[i])

train = [tr_train_pos, tr_train_neg, copy.deepcopy(facts)]
test = [test_pos, test_neg, copy.deepcopy(facts)]

individuals = get_k_best_individuals(best[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'yago2s', 'nell_finances')
    res = test_refine_transfer(kb_target, target, refine, transfer, train, test)
    print(res)
    final_results[f'groot_test:{source}->{target}'].append(res)
    import os
#     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
#     thisFile = f'boostsrl/best/test/results_{target}.db'
#     base = os.path.splitext(thisFile)[0]
#     os.rename(thisFile, base + ".txt")
#     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
#     final = []
#     for i in tt:
#         final.append(i.replace('\n', ''))
#     final_results[f'inf_res:{source}->{target}'].append(final)
    print("===========================================================================")
    
    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.999452, 'AUC PR': 0.999422, 'CLL': -0.434732, 'Precision': [1.0, 0.5], 'Recall': 0.880342, 'F1': 0.936364}, <boostsrl.boostsrl.test object at 0x139404af0>)
INDIVIDUO  2
({'AUC ROC': 0.99947, 'AUC PR': 0.99944, 'CLL': -0.434695, 'Precision': [1.0, 0.5], 'Recall': 0.880342, 'F1': 0.936364}, <boostsrl.boostsrl.test object at 0x139404c10>)
INDIVIDUO  3
({'AUC ROC': 0.999452, 'AUC PR': 0.999422, 'CLL': -0.434711, 'Precision': [1.0, 0.5], 'Recall': 0.880342, 'F1': 0.936364}, <boostsrl.boostsrl.test object at 0x139404ca0>)


In [51]:
final_results[f'brkga_{source}->{target}'] = []
final_results[f'inf_res_brkga:{source}->{target}'] = []
final_results[f'brkga_var_{source}->{target}'] = []
final_results[f'inf_res_brkga_var:{source}->{target}'] = []
save_results(final_results, source, target, 'yago2s', 'nell_finances')

In [52]:
final_results[f'groot_test:{source}->{target}']

[({'AUC ROC': 0.999452,
   'AUC PR': 0.999422,
   'CLL': -0.434732,
   'Precision': [1.0, 0.5],
   'Recall': 0.880342,
   'F1': 0.936364},
  <boostsrl.boostsrl.test at 0x139404af0>),
 ({'AUC ROC': 0.99947,
   'AUC PR': 0.99944,
   'CLL': -0.434695,
   'Precision': [1.0, 0.5],
   'Recall': 0.880342,
   'F1': 0.936364},
  <boostsrl.boostsrl.test at 0x139404c10>),
 ({'AUC ROC': 0.999452,
   'AUC PR': 0.999422,
   'CLL': -0.434711,
   'Precision': [1.0, 0.5],
   'Recall': 0.880342,
   'F1': 0.936364},
  <boostsrl.boostsrl.test at 0x139404ca0>)]

## Mixed

In [62]:
start = time.time()
res = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
ESTOU NESSE CASO
MELHOR RESULTADO:  -0.477144


KeyboardInterrupt: 

In [60]:
aa = test_tree_b(source, target, kb_source, kb_target, structured_src, train, test)
aa

******************************************
Performing Parameter Learning
******************************************
Refine
['worksat(A, B)', {'': 'graduatedfrom(C, B), influences(C, D)', 'true': 'hasacademicadvisor(E, C), hasacademicadvisor(C, F)', 'true,false': 'iscitizenof(C, G), iscitizenof(A, G)', 'true,false,false': 'haswebsite(D, H), diedin(C, I)', 'false': 'hasacademicadvisor(A, J), hasgender(J, K)', 'false,false': 'graduatedfrom(L, B), livesin(L, M)', 'false,false,true': 'iscitizenof(A, M), iscitizenof(L, M)', 'false,false,false': 'hasacademicadvisor(N, A), graduatedfrom(N, P)'}, {'true,true': [0.388, 273, 1209], 'true,false,true': [0.37, 69, 352], 'true,false,false,true': [7.829, 88, 202], 'true,false,false,false': [18.136, 687, 631], 'false,true': [0.451, 865, 344], 'false,false,true,true': [4.969, 33, 98], 'false,false,true,false': [17.103, 703, 501], 'false,false,false,true': [3.67, 85, 16], 'false,false,false,false': [24.03, 1492, 942]}]
['worksat(A, B)', {'': 'graduatedfr

WILL Produced-Tree #1
% FOR companyeconomicsector(A, B):
%   if ( economicsectorcompany(B, C), acquired(D, C) )
%   then if ( bankboughtbank(C, E), bankboughtbank(F, C) )
%   | then return 0.6771965541471302;  // std dev = 3,945, 105,000 (wgt'ed) examples reached here.  /* #neg=19 #pos=86 */
%   | else if ( recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G) )
%   | | then return 0.8348931211460199;  // std dev = 2,965, 387,000 (wgt'ed) examples reached here.  /* #neg=9 #pos=378 */
%   | | else if ( bankchiefexecutiveceo(D, H) )
%   | | | then return -0.1418510649004878;  // std dev = 0,000, 13,000 (wgt'ed) examples reached here.  /* #neg=13 */
%   | | | else return -0.14185106490048793;  // std dev = 0,000, 135,000 (wgt'ed) examples reached here.  /* #neg=135 */
%   else if ( bankboughtbank(I, A), cityhascompanyoffice(J, I) )
%   | then return -0.14185106490048777;  // std dev = 0,000, 5,000 (wgt'ed) examples reached here.  /* #neg=5 */
%   | else if ( economi

Results
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.129455
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total learning time: 34.87 seconds
Total inference time: 6.161 seconds
AUC ROC: 1.0


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.128551
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 9.767 seconds
Parameter learned model CLL: -0.128551


Structure after Parameter Learning
[['companyeconomicsector(A, B)', {'': 'economicsectorcompany(B, C), acquired(D, C)', 'true': 'bankboughtbank(C, E), bankboughtbank(F, C)', 'true,false': 'recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G)', 'true,false,false': 'bankchiefexecutiveceo(D, H)', 'false': 'bankboughtbank(I, A), cityhascompanyoffice(J, I)', 'false,false': 'economicsectorcompany(B, K)', 'false,false,true': 'recursion_companyeconomicsector(A, L), recursion_companyeconomicsector(K, L)'}, 

WILL Produced-Tree #1
% FOR companyeconomicsector(A, B):
%   if ( economicsectorcompany(B, C), acquired(D, C) )
%   then if ( bankboughtbank(C, E), bankboughtbank(F, C) )
%   | then if ( economicsectorcompany(B, A) )
%   | | then return 0.8581489350995121;  // std dev = 0,000, 86,000 (wgt'ed) examples reached here.  /* #pos=86 */
%   | | else return -0.14185106490048777;  // std dev = 0,000, 19,000 (wgt'ed) examples reached here.  /* #neg=19 */
%   | else if ( recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G) )
%   | | then if ( recursion_companyeconomicsector(H, B), companyceo(H, I) )
%   | | | then return 0.8469757507419664;  // std dev = 1,989, 358,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=354 */
%   | | | else return 0.6857351419960634;  // std dev = 2,034, 29,000 (wgt'ed) examples reached here.  /* #neg=5 #pos=24 */
%   | | else return -0.14185106490048804;  // std dev = 0,000, 148,000 (wgt'ed) examples reached here.  /* #neg=148 */
%   else if

Results
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.099913
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total learning time: 182.22 seconds
Total inference time: 4.08 seconds
AUC ROC: 1.0


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.09952
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 6.593 seconds
Refined model CLL: -0.09952


Refining iteration 2
********************************
Candidate for revision
0;;companyeconomicsector(A, B) :- economicsectorcompany(B, C), acquired(D, C).;true;true
0;true;bankboughtbank(C, E), bankboughtbank(F, C).;true;true
0;true,true;economicsectorcompany(B, A).;true;true
0;true,false;recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G).;true;true
0;false;bankboughtbank(J, A), cityhascompanyoffice(K, J).;true;true
0;false,false;recursion_companyeconomicsector(A, B).;true;true
1;;companyeconomicsector(A, B

Results
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.09981
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total learning time: 32.715 seconds
Total inference time: 3.042 seconds
AUC ROC: 1.0


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.099513
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 4.301 seconds
Refined model CLL: -0.099513


Refining iteration 3
********************************
Candidate for revision
0;;companyeconomicsector(A, B) :- economicsectorcompany(B, C), acquired(D, C).;true;true
0;true;bankboughtbank(C, E), bankboughtbank(F, C).;true;true
0;true,true;economicsectorcompany(B, A).;true;true
0;true,false;recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G).;true;true
0;false;bankboughtbank(I, A), cityhascompanyoffice(J, I).;true;true
0;false,false;recursion_companyeconomicsector(A, B).;true;true
1;;companyeconomicsector(A,

Results
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.09981
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total learning time: 34.51 seconds
Total inference time: 6.057 seconds
AUC ROC: 1.0


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.099513
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 11.195 seconds
Refined model CLL: -0.099513


******************************************
Best model found
******************************************
Results
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.09981
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total learning time: 32.715 seconds
Total inference time: 3.042 seconds
AUC ROC: 1.0
Total revision time: 316.17099999999994
Best scored revision CLL: -0.099513




 {'AUC ROC': 1.0,
  'AUC PR': 1.0,
  'CLL': -0.09981,
  'Precision': [1.0, 0.5],
  'Recall': 1.0,
  'F1': 1.0,
  'Learning time': 32.715,
  'Inference time': 3.042},
 [['companyeconomicsector(A, B)',
   {'': 'economicsectorcompany(B, C), acquired(D, C)',
    'true': 'bankboughtbank(C, E), bankboughtbank(F, C)',
    'true,true': 'economicsectorcompany(B, A)',
    'true,false': 'recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G)',
    'true,false,true': 'companyceo(A, H)',
    'false': 'bankboughtbank(I, A), cityhascompanyoffice(J, I)',
    'false,false': 'recursion_companyeconomicsector(A, B)'},
   {'true,true,true': [0.0, 0, 86],
    'true,true,false': [0.0, 19, 0],
    'true,false,true,true': [1.684, 3, 52],
    'true,false,true,false': [2.427, 6, 326],
    'true,false,false': [0.0, 148, 0],
    'false,true': [0.0, 5, 0],
    'false,false,true': [0.0, 0, 80],
    'false,false,false': [2.48e-07, 363, 0]}],
  ['companyeconomicsector(A, B)',
   {'': 'economicsec

In [61]:
background_knowledge_src = boostsrl.modes(kb_target, [target], useStdLogicVariables=False, 
                                      maxTreeDepth=3, nodeSize=2, numOfClauses=8)
sys.path.insert(0, '../TreeBoostler')

from revision import revision

def print_function(message):
        global experiment_title
        global nbr
        experiment_title = f'Twitter->Yeast2: {source}->{target}'
        nbr = 1
        if not os.path.exists('experiments/' + experiment_title):
            os.makedirs('experiments/' + experiment_title)
        with open('experiments/' + experiment_title + '/' + str(nbr) + '_' + experiment_title + '.txt', 'a') as f:
            print(message, file=f)
            print(message)

model = revision.learn_test_model(background_knowledge_src, boostsrl, target, tr_train_pos, tr_train_neg, copy.deepcopy(facts), test_pos, test_neg, copy.deepcopy(facts), trees=10, print_function=print_function)

WILL Produced-Tree #1
% FOR companyeconomicsector(A, B):
%   if ( recursion_companyeconomicsector(A, B) )
%   then return 0.858148935099508;  // std dev = 2,57e-06, 544,000 (wgt'ed) examples reached here.  /* #pos=544 */
%   else return -0.14185106490048816;  // std dev = 0,000, 544,000 (wgt'ed) examples reached here.  /* #neg=544 */
WILL Produced-Tree #2
% FOR companyeconomicsector(A, B):
%   if ( recursion_companyeconomicsector(A, B) )
%   then return 0.7194734122109585;  // std dev = 0,000, 544,000 (wgt'ed) examples reached here.  /* #pos=544 */
%   else return -0.12544463852839233;  // std dev = 0,000, 544,000 (wgt'ed) examples reached here.  /* #neg=544 */
WILL Produced-Tree #3
% FOR companyeconomicsector(A, B):
%   if ( recursion_companyeconomicsector(A, B) )
%   then return 0.555366436746283;  // std dev = 8,09e-07, 544,000 (wgt'ed) examples reached here.  /* #pos=544 */
%   else return -0.1123163781936057;  // std dev = 0,000, 544,000 (wgt'ed) examples reached here.  /* #neg=54